# Revised case normalization for Hirslanden St. Anna 2017

Notes:

- Inserted 71 cases into the DB


In [1]:
import sys
sys.path.insert(0, '/home/jovyan/work')

from src.revised_case_normalization.notebook_functions.global_configs import *
from src.revised_case_normalization.notebook_functions.normalize import normalize
from src.revised_case_normalization.notebook_functions.revise import revise
from src.revised_case_normalization.notebook_functions.group import group
from src.revised_case_normalization.notebook_functions.update_db import update_db

In [2]:
file_info = FileInfo(os.path.join(ROOT_DIR, 'raw_data/HI_Aarau_Birshof_ST. Anna.xlsx'),
                     'Hirslanden St. Anna', '2017', 'Änderungen_ST. Anna_2017')
print(file_info)


FileInfo(path='/home/jovyan/work/src/revised_case_normalization/raw_data/HI_Aarau_Birshof_ST. Anna.xlsx', hospital_name_db='Hirslanden St. Anna', year='2017', sheet='Änderungen_ST. Anna_2017')


In [3]:
revised_cases_df = normalize(file_info)
revised_cases_df

2022-11-14 09:41:41.780 | INFO     | src.revised_case_normalization.notebook_functions.normalize:normalize:35 - Read 72 cases for Hirslanden St. Anna 2017
2022-11-14 09:41:41.785 | INFO     | src.revised_case_normalization.notebook_functions.normalize:normalize:63 - TYPES:
tranche                   string
datum                     string
case_id                   object
patient_id                object
gender                    string
age_years                  int64
bfs_code                  string
duration_of_stay           int64
pflegetage neu            string
pccl                       int64
pccl neu                  string
old_pd                    string
primary_diagnosis         string
added_icds                string
removed_icds              string
added_chops               string
removed_chops             string
drg                       string
drg neu                   string
cw alt                    string
cw neu                    string
cw-änderung möglich       string


,case_id,patient_id,gender,age_years,duration_of_stay,case_id_norm,old_pd,primary_diagnosis,bfs_code,pccl,drg,added_icds,removed_icds,added_chops,removed_chops
0,0009243576,4A3BCA50B474DD04,M,74,29,9243576,M310,M310,M950,3,I66B,[D508],[D648],[],[]
1,0009245400,18E7929A20616AF8,M,74,22,9245400,K758,K758,M100,3,H63C,"[D508, I653]",[],[],[]
2,0009247208,43053E793FC348BF,W,77,7,9247208,N823,N823,M200,3,G18B,[K564],[],[],[]
3,0009248506,463BA4104B2B5C4E,W,83,25,9248506,M4806,M4806,M200,3,I10C,[E43],[],[890A32],[]
4,0009248541,FDEDA859DAB42487,M,78,12,9248541,J180,J180,M100,3,E40C,"[I652, F019]",[],[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,0009339263,2ABB14A8F64FAB70,M,69,2,9339263,K227,K227,M100,2,G67D,[J9600],[],[],[]
68,0009340255,516515D782F30461,M,62,3,9340255,K703,K703,M100,3,H62B,[R18],[],[],[]
69,0009344799,E968664B83D26A1F,W,84,8,9344799,F03,F03,M100,3,B63B,[J9600],[],[],[]
70,0009359214,EAC9772C144152EB,M,59,5,9359214,C187,C187,M200,2,G17Z,[F101],[],[],[]


# Match revised DtoD data with the BfS data from the database

In [4]:
cols_to_join = list(VALIDATION_COLS)
cols_to_join.remove(CASE_ID_COL)
cols_to_join.append(NORM_CASE_ID_COL)
# Ignore the patient ID in this dataset, because it is encrypted
cols_to_join.remove(PATIENT_ID_COL)

revised_cases, unmatched = revise(file_info, revised_cases_df, validation_cols=cols_to_join)
revised_cases

2022-11-14 09:41:42.766 | INFO     | src.service.bfs_cases_db_service:get_sociodemographics_for_hospital_year:90 - Read 12547 rows from the DB, for the hospital 'Hirslanden St. Anna' in 2017
2022-11-14 09:41:42.778 | WARNING  | src.revised_case_normalization.notebook_functions.revise:revise:47 - 1 rows could not be matched, given ['age_years', 'case_id_norm', 'duration_of_stay', 'gender']


,aimedic_id,case_id,primary_diagnosis,secondary_diagnoses,primary_procedure,secondary_procedures,gender,age_years,age_days,gestation_age,duration_of_stay,ventilation_hours,grouper_admission_type,admission_date,admission_weight,grouper_discharge_type,discharge_date
0,33614,0009219917,M4856,"[M4786, M5326, E1120, N298, N182, ...]",8108::20170206,"[8138::20170206, 816310::20170206, 030936::20170206, 030991::20170206, 845110::20170206, ...]",W,71,0,0,9,0,01,2017-02-05,0,00,2017-02-14
1,33655,0009340255,K703,"[I982, N390, B962, I1090, F101, ...]",5491::20171011,"[889717::20171012, 451310::20171012]",M,62,0,0,3,0,01,2017-10-11,0,00,2017-10-14
2,33662,0009291492,I7021,"[I340, I361, I480, Z921, I2529, ...]",395019:R:20170530,"[0041:R:20170530, 004B18:R:20170530, 0045:R:20170530, 005599:R:20170530, 004D::20170530]",M,95,0,0,2,0,01,2017-05-29,0,00,2017-05-31
3,33807,0009323466,N12,"[B962, A418, R650, I5014, I5001, ...]",992909::20170820,[948X40::20170822],W,77,0,0,8,0,01,2017-08-20,0,00,2017-08-28
4,33958,0009339263,K227,"[D508, D62, K3181, Z867, I2728, ...]",423329::20171006,"[990410::20171006, 992909::20171006]",M,69,0,0,2,0,01,2017-10-05,0,00,2017-10-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66,43580,0009328159,N10,"[B962, A4151, R650, I2519, Z951, ...]",992213::20170902,[890A32::],M,83,0,0,9,0,01,2017-09-02,0,00,2017-09-11
67,43754,0009247208,N823,"[K9183, C20, Z432, Z926, Z923, ...]",707399::20170105,"[4879::20170105, 703399::20170105, 009910::20170105]",W,77,0,0,7,0,01,2017-01-04,0,00,2017-01-11
68,44360,0009250270,C541,"[C778, C780, G405, G478, E440, ...]",921905::20170105,"[948X40::20170106, 890A32::20170108]",W,88,0,0,14,0,01,2017-01-04,0,00,2017-01-18
69,44849,0009267839,C61,"[K660, C775, I480, Z921, K4091, ...]",605X10::20170421,"[009950::20170421, 403X23:B:20170421, 99B811::20170421, 530711:R:20170421]",M,73,0,0,7,0,01,2017-04-20,0,00,2017-04-27


In [5]:
if unmatched.shape[0] > 0:
    display(unmatched)

,case_id,patient_id,gender,age_years,duration_of_stay,case_id_norm,old_pd,primary_diagnosis,bfs_code,pccl,drg,added_icds,removed_icds,added_chops,removed_chops,aimedic_id,case_id_db,patient_id_db,age_days,admission_weight,gestation_age,admission_date,grouper_admission_type,discharge_date,grouper_discharge_type,ventilation_hours
36,0009344274,FBF3F2F3AC175A05,W,67,9,9344274,I5014,I5014,M100,3,F24B,"[E1191, I340]",[E1190],[],[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
revisions_update, diagnoses_update, procedures_update = group(revised_cases)
revisions_update[REVISION_DATE_COL] = '2022-12-31'

2022-11-14 09:41:43.147 | INFO     | src.revised_case_normalization.notebook_functions.group:group:83 - Grouping 71 cases ...
2022-11-14 09:41:44.109 | SUCCESS  | src.revised_case_normalization.notebook_functions.group:group:88 - Grouped 71 cases into: 71 revisions, 633 diagnoses rows, 290 procedure rows


In [7]:
revisions_update

,aimedic_id,drg,drg_cost_weight,effective_cost_weight,pccl,revision_date
0,33614,I06C,4.04,4.04,4,2022-12-31
1,33655,H60Z,1.48,1.12,4,2022-12-31
2,33662,F14A,3.47,1.95,4,2022-12-31
3,33807,L63B,1.04,1.04,4,2022-12-31
4,33958,G50Z,1.35,0.93,3,2022-12-31
...,...,...,...,...,...,...
66,43580,L63B,1.04,1.04,4,2022-12-31
67,43754,G13Z,3.96,3.96,4,2022-12-31
68,44360,N60A,1.51,1.51,4,2022-12-31
69,44849,M01A,3.01,3.01,4,2022-12-31


In [8]:
update_db(revisions_update, diagnoses_update, procedures_update)

2022-11-14 09:41:55.348 | INFO     | src.service.bfs_cases_db_service:insert_revised_cases_into_revisions:249 - Trying to insert 71 cases into the 'Revisions' table ...
2022-11-14 09:41:55.814 | INFO     | src.service.bfs_cases_db_service:insert_revised_cases_into_revisions:275 - Deleted 71 rows from the "Revisions" table, which is about to be updated
2022-11-14 09:41:55.884 | SUCCESS  | src.service.bfs_cases_db_service:insert_revised_cases_into_revisions:286 - Inserted 71 cases into the 'Revisions' table
2022-11-14 09:41:55.887 | INFO     | src.service.bfs_cases_db_service:insert_revised_cases_into_diagnoses:298 - Trying to insert 633 rows into the 'Diagnoses' table ...
2022-11-14 09:41:57.351 | SUCCESS  | src.service.bfs_cases_db_service:insert_revised_cases_into_diagnoses:336 - Inserted 633 rows into the 'Diagnoses' table
2022-11-14 09:41:57.352 | INFO     | src.service.bfs_cases_db_service:insert_revised_cases_into_procedures:347 - Trying to insert 290 rows into the 'Procedures' ta